In [33]:
import pandas as pd
import boto3
import json
import psycopg2

In [34]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','key')
SECRET = config.get('AWS','secret')


In [8]:
s3 = boto3.resource('s3',
                    region_name='us-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                  )

bucket = s3.Bucket('udacity-dend')

#  Display data 
## Song Files

In [ ]:
for obj in bucket.objects.filter(Prefix='song_data'):
    print(obj)

## Log Files

In [ ]:
for obj in bucket.objects.filter(Prefix='log_data'):
    print(obj)

## Log Json file

In [29]:
content_object = s3.Object('udacity-dend', 'log_json_path.json')
file_content = content_object.get()['Body'].read().decode('utf-8')

print(file_content)


{
    "jsonpaths": [
        "$['artist']",
        "$['auth']",
        "$['firstName']",
        "$['gender']",
        "$['itemInSession']",
        "$['lastName']",
        "$['length']",
        "$['level']",
        "$['location']",
        "$['method']",
        "$['page']",
        "$['registration']",
        "$['sessionId']",
        "$['song']",
        "$['status']",
        "$['ts']",
        "$['userAgent']",
        "$['userId']"
    ]
}


# Display DB Tables

In [48]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()


In [52]:
cur.execute("""
SELECT * FROM staging_songs
Limit 5;
"""
)
print(cur.fetchall()) 

[('SONHGLD12AB0188D47', 1, 'AR1S3NH1187B98C2BC', None, None, 'Clarks Town, Jamaica', 'Anthony B', 'Our Father', 202.4224, 1999), ('SOQXJID12A8AE456CD', 1, 'ARBJ3VU1187B9B472D', None, None, '', 'Love Of Lesbian', 'Domingo astronómico', 264.69832, 2005), ('SOAFBKM12AB01837A7', 1, 'ARL14X91187FB4CF14', None, None, '', 'Anal Cunt', 'Brain Dead', 94.22322, 1995), ('SOINBCU12A6D4F94C0', 1, 'ARV1JVD1187B9AD195', '35.914630000000002', '-79.056610000000006', 'Chapel Hill, NC', 'Loudon Wainwright III', 'Human Cannonball', 190.48444, 1995), ('SODJYTL12A8C14550E', 1, 'ARHAJH611C8A4219F1', None, None, '', "La Bottega Dell'Arte", 'Noi Nel Bene_ Noi Nel Male', 223.242, 0)]


## Testing

In [13]:
print(
    """
    copy         staging_events 
    from         {}
    credentials  'aws_iam_role={}'
    region       'us-west-2'
    json         {};
    """.format(config['S3']['LOG_DATA'], 
               config['IAM']['IAM_ARN'], 
               config['S3']['LOG_JSONPATH'])
)


    copy         staging_events 
    from         's3://udacity-dend/log_data'
    credentials  'aws_iam_role=arn:aws:iam::567714418050:role/redRole'
    region       'us-west-2'
    json         's3://udacity-dend/log_json_path.json';
    
